In [2]:
import numpy as np
import pandas as pd
from ase import Atoms
from ase import neighborlist
from ase import units
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.md.verlet import VelocityVerlet
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.io.extxyz import read_extxyz, write_extxyz
import statsmodels.api as sm

In [5]:
def create_VAR_OLS_model(main, secondary, order=2):
    input = len(main)-order
    num_secondaries = secondary.shape[0]
    X = np.zeros([input,order*(num_secondaries+1)+1])
    for i in range(order):
        X[:,i*(num_secondaries+1)] = main[i:input+i]
        for j in range(num_secondaries):
            X[:,i*(num_secondaries+1)+j+1] = secondary[j,i:input+i]
    X[:,-1] = np.ones(len(main)-order)
    return sm.OLS(main[order:], X)
pred_step = 4
input = 6
#Load molecule data
mol = read(r"C:\Users\birk\OneDrive - University of Copenhagen\Documents\KU tid\Gemma_group\PUK\Python_scripts\Data\EtOH_moldyn_xyz_test.xyz", index=':'+str(input))

print(mol[0].get_forces())
# for different atoms
    #Create model
    #Predict the next steps
#Create array with all forces
forces = np.zeros((mol[0].get_forces().shape[0], mol[0].get_forces().shape[1], input))
for i in range(input):
    print(mol[i].get_forces())
    print(i)
    forces[:,:,i] = mol[i].get_forces()
    print(forces[:,:,i])

pred_forces = np.zeros((forces.shape[0], forces.shape[1], pred_step))

n=0
def write_to_xyz(a=mol):
    if n == 0:
        write("EtOH_moldyn_test.xyz", a, append=True)
        n += 1
    else:
        write("EtOH_moldyn_test.xyz", a[-1], append=True)




    



ValueError: buffer is smaller than requested size